<a href="https://colab.research.google.com/github/Jeevesh8/arg_mining/blob/main/experiments/long_context_am.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Dependencies

In [ ]:
%%capture
#if running on colab, install below 4
#!git clone https://github.com/Jeevesh8/arg_mining
#!pip install transformers
#!pip install seqeval datasets allennlp
#!pip install flax

#if connected to local runtime, run the next command too
#pip install bs4 tensorflow torch 



*   Update ``arg_mining/datasets/cmv_modes/configs.py`` as per your requirements, all experiments considered till now, set ``batch_size`` to 2, and all other variables with their default value.



In [1]:
#Run to ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Load Metric

In [2]:
from datasets import load_metric
metric = load_metric('seqeval')

### Define & Load Tokenizer, Model, Dataset

In [3]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda', index=0)

In [5]:
model_version = 'allenai/longformer-base-4096'

In [6]:
%%capture
from transformers import LongformerTokenizer, AutoModel
tokenizer = LongformerTokenizer.from_pretrained(model_version)
transformer_model = AutoModel.from_pretrained(model_version).to(device)

In [13]:
import torch.nn as nn

#Extend Token Type Embeddings
def resize_token_type_embeddings(transformer_model, new_size):
    old_embeddings = transformer_model.embeddings.token_type_embeddings.weight
    old_size, hidden_dim = old_embeddings.shape
    transformer_model.embeddings.token_type_embeddings = nn.Embedding(new_size, hidden_dim, device=transformer_model.device)
    with torch.no_grad():
        transformer_model.embeddings.token_type_embeddings.weight[:old_size] = old_embeddings

resize_token_type_embeddings(transformer_model, 2)

In [8]:
with open('./Discourse_Markers.txt') as f:
    discourse_markers = [dm.strip() for dm in f.readlines()]

In [9]:
%%capture
from arg_mining.datasets.cmv_modes import load_dataset, data_config

tokenizer.add_tokens(data_config["special_tokens"])

transformer_model.resize_token_embeddings(len(tokenizer))

def get_datasets():
    train_dataset, valid_dataset, test_dataset = load_dataset(tokenizer=tokenizer,
                                                              train_sz=80,
                                                              test_sz=20,
                                                              mask_tokens=discourse_markers)
    return train_dataset, valid_dataset, test_dataset

### Define layers for a Linear-Chain-CRF

In [10]:
from allennlp.modules.conditional_random_field import ConditionalRandomField as crf

ac_dict = data_config["arg_components"]

allowed_transitions =([(ac_dict["B-C"], ac_dict["I-C"]), 
                       (ac_dict["B-P"], ac_dict["I-P"])] + 
                      [(ac_dict["I-C"], ac_dict[ct]) 
                        for ct in ["I-C", "B-C", "B-P", "O"]] +
                      [(ac_dict["I-P"], ac_dict[ct]) 
                        for ct in ["I-P", "B-C", "B-P", "O"]] +
                      [(ac_dict["O"], ac_dict[ct]) 
                        for ct in ["O", "B-C", "B-P"]])
                    
linear_layer = nn.Linear(transformer_model.config.hidden_size,
                         len(ac_dict)).to(device)

crf_layer = crf(num_tags=len(ac_dict),
                constraints=allowed_transitions,
                include_start_end_transitions=False).to(device)

cross_entropy_layer = nn.CrossEntropyLoss(weight=torch.log(torch.tensor([3.3102, 61.4809, 3.6832, 49.6827, 2.5639], 
                                                                        device=device)), reduction='none')

### Global Attention Mask Utility for Longformer

In [11]:
import numpy as np
from threading import Lock

def get_global_attention_mask(tokenized_threads: np.ndarray) -> np.ndarray:
    """Returns an attention mask, with 1 where there are [USER{i}] tokens and 
    0 elsewhere.
    """
    mask = np.zeros_like(tokenized_threads)
    for user_token in ["UNU"]+[f"[USER{i}]" for i in range(data_config["max_users"])]:
        user_token_id = tokenizer.encode(user_token)[1:-1]
        mask = np.where(tokenized_threads==user_token_id, 1, mask)
    return np.array(mask, dtype=bool)

### Loss and Prediction Function

In [12]:
from typing import Tuple

In [14]:
def compute(batch: Tuple[torch.Tensor, torch.Tensor, torch.Tensor],
            preds: bool=False, cross_entropy: bool=True):
    """
    Args:
        batch:  A tuple having tokenized thread of shape [batch_size, seq_len],
                component type labels of shape [batch_size, seq_len], and a global
                attention mask for Longformer, of the same shape.
        
        preds:  If True, returns a List(of batch_size size) of Tuples of form 
                (tag_sequence, viterbi_score) where the tag_sequence is the 
                viterbi-decoded sequence, for the corresponding sample in the batch.
        
        cross_entropy:  This argument will only be used if preds=False, i.e., if 
                        loss is being calculated. If True, then cross entropy loss
                        will also be added to the output loss.
    
    Returns:
        Either the predicted sequences with their scores for each element in the batch
        (if preds is True), or the loss value summed over all elements of the batch
        (if preds is False).
    """
    tokenized_threads, token_type_ids, comp_type_labels, global_attention_mask = batch
    
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,
                               attention_mask=pad_mask,
                               global_attention_mask=global_attention_mask).last_hidden_state)
    
    if preds:
        return crf_layer.viterbi_tags(logits, pad_mask)
    
    log_likelihood = crf_layer(logits, comp_type_labels, pad_mask)
    
    if cross_entropy:
        logits = logits.reshape(-1, logits.shape[-1])
        
        pad_mask, comp_type_labels = pad_mask.reshape(-1), comp_type_labels.reshape(-1)
        
        ce_loss = torch.sum(pad_mask*cross_entropy_layer(logits, comp_type_labels))
        
        return ce_loss - log_likelihood

    return -log_likelihood

### Define optimizer

In [15]:
from itertools import chain

import torch.optim as optim

optimizer = optim.Adam(params = chain(transformer_model.parameters(),
                                      linear_layer.parameters(),
                                      crf_layer.parameters()),
                       lr = 2e-5,)

### Training And Evaluation Loops

In [16]:
def train(dataset):
    accumulate_over = 4
    
    optimizer.zero_grad()

    for i, (tokenized_threads, masked_threads, comp_type_labels, _ ) in enumerate(dataset):
        global_attention_mask = torch.tensor(get_global_attention_mask(tokenized_threads),
                                             device=device, dtype=torch.int32)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                         device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                      device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device, dtype=torch.long)
        
        global_attention_mask = torch.squeeze(global_attention_mask, dim=0)
        
        loss = compute((tokenized_threads,
                        torch.where(masked_threads==tokenizer.mask_token_id, 1, 0), 
                        comp_type_labels, 
                        global_attention_mask))/data_config["batch_size"]
        
        print("Loss: ", loss)
        loss.backward()
        
        if i%accumulate_over==accumulate_over-1:
            optimizer.step()
            optimizer.zero_grad()
    
    optimizer.step()

In [17]:
transformer_model.config.type_vocab_size = 2

In [18]:
def evaluate(dataset, metric):
    
    int_to_labels = {v:k for k, v in ac_dict.items()}
    
    for tokenized_threads, masked_threads, comp_type_labels, _ in dataset:
        
        global_attention_mask = torch.tensor(get_global_attention_mask(tokenized_threads), 
                                             device=device)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0),
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0),
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0),
                                        device=device)
        global_attention_mask = torch.squeeze(global_attention_mask, dim=0)
        
        preds = compute((tokenized_threads,
                         torch.where(masked_threads==tokenizer.mask_token_id, 1, 0), 
                         comp_type_labels,
                         global_attention_mask),
                        preds=True)
        
        lengths = torch.sum(torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0), 
                            axis=-1)
        
        preds = [ [int_to_labels[pred] for pred in pred[0][:lengths[i]]]
                  for i, pred in enumerate(preds)
                ]
        
        refs = [ [int_to_labels[ref] for ref in labels[:lengths[i]]]
                 for i, labels in enumerate(comp_type_labels.cpu().tolist())
               ]
        
        metric.add_batch(predictions=preds, 
                         references=refs)
    
    print(metric.compute())

### Final Training

In [19]:
n_epochs = 35

In [20]:
for epoch in range(n_epochs):
    print(f"------------EPOCH {epoch+1}---------------")
    train_dataset, _, test_dataset = get_datasets()
    train(train_dataset)
    evaluate(test_dataset, metric)

------------EPOCH 1---------------


fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.
2021-08-26 10:43:21.880074: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-08-26 10:43:21.880870: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Loss:  tensor(3273.7771, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2606.2212, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3360.3765, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3211.7551, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2050.5786, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2062.1758, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2562.0024, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2806.7324, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1786.5348, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1887.1169, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2070.9905, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3208.7817, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2549.1536, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3717.8696, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3535.7019, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  ten

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(2069.5649, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1717.7789, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2256.8770, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2296.1519, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1589.2266, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1561.5105, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1812.2649, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1745.1355, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1230.9990, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1270.0487, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1354.4434, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2230.5303, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1889.0938, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3188.2751, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2685.1611, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  ten

...

Loss:  tensor(10.1082, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.9524, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.7584, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.9597, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(9.4076, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(10.0801, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.8489, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(5.1730, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(4.3108, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3.2192, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.2632, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14.2010, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.0073, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(25.9204, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(23.5888, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.0804, device='cuda:0', grad_fn

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(9.8287, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.7227, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.0868, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.0237, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.5525, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(9.7636, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.5108, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(4.8563, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(4.0557, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3.9324, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.0698, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(13.4509, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.5055, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(23.0086, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(23.1315, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.4623, device='cuda:0', grad_fn=<

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(9.6000, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.9377, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(9.6775, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(10.5700, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.9922, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(10.0189, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.0766, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(5.2239, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3.3332, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2.7011, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.7920, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.9144, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.4337, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(20.3117, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(22.9691, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.2325, device='cuda:0', grad_fn=<D

### Rough -- Checking dataset

In [11]:
def get_datasets():
    train_dataset, valid_dataset, test_dataset = load_dataset(tokenizer=tokenizer,
                                                              train_sz=80,
                                                              test_sz=20,
                                                              mask_tokens=discourse_markers)
    return train_dataset, valid_dataset, test_dataset

In [12]:
train_dataset, _, test_dataset = get_datasets()

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.
2021-08-26 07:47:02.997781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-08-26 07:47:02.998787: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
for tokenized_threads, masked_threads, comp_type_labels, _ in test_dataset:
    tokenized_threads, masked_threads, comp_type_labels = tokenized_threads[0], masked_threads[0], comp_type_labels[0]
    for tokenized_thread, masked_thread, comp_type_label in zip(tokenized_threads, masked_threads, comp_type_labels):
        print(comp_type_label[:100])
        print(tokenized_thread[:100])
        print(tokenizer.decode(tokenized_thread[:100]))
        start, end = 0, 0
        prev_type = "other"
        i = 0
        while i<tokenized_thread.shape[0]:
            if comp_type_label[i]==ac_dict["O"]:
                if prev_type=="other":
                    end += 1
                else:
                    print("Component: ", tokenizer.decode(tokenized_thread[start:end+1]), " of type: ", prev_type, tokenized_thread[start:end+1])
                    print("Masked Component: ", tokenizer.decode(masked_thread[start:end+1]), " of type: ", prev_type, masked_thread[start:end+1])
                    start = i
                    end = i
                    prev_type="other"
                
            if comp_type_label[i] in [ac_dict["B-C"], ac_dict["B-P"]]:
                print("Component: ", tokenizer.decode(tokenized_thread[start:end+1]), " of type: ", prev_type, tokenized_thread[start:end+1])
                print("Masked Component: ", tokenizer.decode(masked_thread[start:end+1]), " of type: ", prev_type, masked_thread[start:end+1])
                start = i
                end = i
                prev_type = "Claim" if comp_type_label[i]==ac_dict["B-C"] else "Premise"
            
            if comp_type_label[i] in [ac_dict["I-C"], ac_dict["I-P"]]:
                end += 1
            
            i+=1
        break
    break

[0 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 2 2 2 2 2 0 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2]
[    0 18814   846    35  7978     9  1901    16   145   551   350   444
 50269  1437    96     5    94   367   688    52   348    56    80  1307
  1061  1369    11     5   232     6  1437   258     9    61    58  1726
    30  3510  8941     7  1437    22  3519     9  1901     4  1437    22
  1437    20    78   145     5 11597     9  6366    81    20 21902     6
  1437     8   452     5  1094    23     5  4088     9    10 33937  4320
    11  2201     4  1437  1437  1437   100  1819   923    84   481  1901
  1437    53     7   162  1437  1437  8585    16    10   699   516   227
 20203   110    78  8322]
<s>CMV: Freedom of speech is being taken too far [USER0]   In the last few weeks we've had two huge events happen in the world,  both of which were caused by matters relating to  " freed